# 示例酒店和航班预订代理

此解决方案将帮助您预订机票和酒店。场景是一次旅行：从伦敦希思罗机场（LHR）出发，2024年2月20日飞往纽约肯尼迪机场（JFK），2025年2月27日返回，仅搭乘英国航空公司的经济舱。我希望入住纽约的希尔顿酒店，请提供航班和酒店的费用。


# 初始化 Azure AI Agent 服务并从 **.env** 获取配置信息

### **.env**

创建一个 .env 文件

**.env** 包含 Azure AI Agent 服务的连接字符串、AOAI 使用的模型，以及对应的 Google API 搜索服务 API、ENDPOINT 等信息。

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "您的 Azure AI Agent 服务模型部署名称"

[**NOTE**] 您需要一个具有 100,000 速率限制（每分钟令牌数）和 600 速率限制（每分钟请求数）的模型。

  您可以在 Azure AI Foundry 的模型和端点中获取模型。

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "您的 Azure AI Agent 服务项目连接字符串"

  您可以在 AI Foundry 门户屏幕的项目概览中获取项目连接字符串。

- **SERPAPI_SEARCH_API_KEY** = "您的 SERPAPI 搜索 API KEY"
- **SERPAPI_SEARCH_ENDPOINT** = "您的 SERPAPI 搜索 ENDPOINT"

要获取 Azure AI Agent 服务的模型部署名称和项目连接字符串，您需要创建 Azure AI Agent 服务。建议直接使用[此模板](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json)进行创建。（***注意：*** Azure AI Agent 服务目前仅在有限的区域设置。建议参考[此链接](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support)设置区域）

Agent 需要访问 SERPAPI。建议通过[此链接](https://serpapi.com/searches)注册。注册后，您可以获得唯一的 API KEY 和 ENDPOINT。


# 设置

要运行此笔记本，您需要确保通过运行 `pip install -r requirements.txt` 安装了所需的库。


In [ ]:
from semantic_kernel import __version__

__version__

您的语义内核版本应至少为1.27.2。


加载您的 .env 文件设置和资源，请确保您已添加密钥和设置并创建了本地 .env 文件。


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# 登录到 Azure

现在您需要登录到 Azure。打开终端并运行以下命令：

```bash
az login
```

此命令将提示您输入 Azure 凭据，从而使 Azure AI Agent 服务能够正常运行。


# 说明：
这是一个变量，用于存储访问 SERP（搜索引擎结果页面）API 服务的 API 密钥。API 密钥是用于验证与您账户相关的请求的唯一标识符。

目的：这行代码的目的是将 API 密钥存储在一个变量中，以便用于验证对 SERP API 服务的请求。访问该服务并执行搜索操作需要提供 API 密钥。
如何获取 SERP API 密钥：要获取 SERP API 密钥，请按照以下一般步骤操作：https://serpapi.com（具体步骤可能因您使用的 SERP API 服务而异）：

选择一个 SERP API 服务：目前有多种 SERP API 服务可供选择，例如 SerpAPI、Google Custom Search JSON API 等。选择最适合您需求的服务。

注册账户：访问所选 SERP API 服务的网站并注册账户。您可能需要提供一些基本信息并验证您的电子邮件地址。

创建 API 密钥：注册完成后，登录您的账户，进入 API 部分或控制面板。查找创建或生成新 API 密钥的选项。
将 API 密钥复制到您的 .env 文件中。


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# 解释：
BASE_URL：这是一个变量，用于存储SERP API端点的基础URL。变量名BASE_URL是一种约定，用来表明这个URL是发起API请求的起点。  
'https://serpapi.com/search'：  

这是分配给BASE_URL变量的实际URL字符串。它表示使用SERP API执行搜索查询的端点。

# 目的：
这一行代码的目的是定义一个常量，用于保存SERP API的基础URL。这个URL将作为构建API请求以执行搜索操作的起点。

# 用法：
通过将基础URL定义在一个变量中，你可以在代码中任何需要向SERP API发起请求的地方轻松复用它。这使得代码更易于维护，并减少了在多个地方硬编码URL而导致错误的风险。当前的示例是https://serpapi.com/search?engine=bing，它使用的是Bing搜索API。可以在https://Serpapi.com选择不同的API。


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# 说明：

这是插件代码所在的位置。

类定义：`class BookingPlugin`：定义了一个名为 BookingPlugin 的类，包含用于预订酒店和航班的方法。

酒店预订方法：

- `@kernel_function(description="booking hotel")`：一个装饰器，用于描述该函数是一个用于预订酒店的内核函数。
- `def booking_hotel(self, query: Annotated[str, "城市名称"], check_in_date: Annotated[str, "酒店入住时间"], check_out_date: Annotated[str, "酒店退房时间"]) -> Annotated[str, "返回酒店预订信息的结果"]:`：定义了一个用于预订酒店的方法，带有注解的参数和返回类型。

该方法构建了一个用于酒店预订请求的参数字典，并向 SERP API 发送 GET 请求。它检查响应状态，如果成功则返回酒店属性信息，如果请求失败则返回 None。

航班预订方法：

- `@kernel_function(description="booking flight")`：一个装饰器，用于描述该函数是一个用于预订航班的内核函数。
- `def booking_flight(self, origin: Annotated[str, "出发地名称"], destination: Annotated[str, "目的地名称"], outbound_date: Annotated[str, "出发日期"], return_date: Annotated[str, "返程日期"]) -> Annotated[str, "返回航班预订信息的结果"]:`：定义了一个用于预订航班的方法，带有注解的参数和返回类型。

该方法分别构建了用于出发航班和返程航班请求的参数字典，并向 SERP API 发送 GET 请求。它检查响应状态，如果成功则将航班信息追加到结果字符串中，如果请求失败则打印错误信息。该方法返回包含航班信息的结果字符串。


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# 解释：
导入语句：导入用于 Azure 凭据、AI 代理、聊天消息内容、作者角色以及内核函数装饰器的必要模块。

异步上下文管理器：`async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`：设置一个异步上下文管理器，用于处理 Azure 凭据并创建一个 AI 代理客户端。

代理名称和指令：
- `AGENT_NAME = "BookingAgent"`：定义代理的名称。
- `AGENT_INSTRUCTIONS = """..."""`：为代理提供详细的指令，说明如何处理预订请求。

创建代理定义：`agent_definition = await client.agents.create_agent(...)`：使用指定的模型、名称和指令创建一个代理定义。

创建 AzureAI 代理：`agent = AzureAIAgent(...)`：使用客户端、代理定义和定义的插件创建一个 AzureAI 代理。

创建线程：`thread: AzureAIAgentThread | None = None`：为代理创建一个线程。并不需要先创建线程——如果提供的值为 `None`，将在首次调用时创建一个新线程，并作为响应的一部分返回。

用户输入：`user_inputs = ["..."]`：定义一个用户输入列表，供代理处理。

在 `finally` 块中，删除线程和代理以清理资源。


# 身份验证

`DefaultAzureCredential` 类是 Azure SDK for Python 的一部分。它提供了一种默认方式来与 Azure 服务进行身份验证。该类会按照特定的顺序尝试多种身份验证方法，例如环境变量、托管身份以及 Azure CLI 凭据。

异步操作：aio 模块表明 DefaultAzureCredential 类支持异步操作。这意味着你可以将其与 asyncio 一起使用，以执行非阻塞的身份验证请求。


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**免责声明**：  
本文档使用AI翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 进行翻译。尽管我们努力确保翻译的准确性，但请注意，自动翻译可能包含错误或不准确之处。原始语言的文档应被视为权威来源。对于关键信息，建议使用专业人工翻译。我们不对因使用此翻译而产生的任何误解或误读承担责任。
